<a href="https://colab.research.google.com/github/f247805/ABSA-Analysis-with-Explainability-Methods/blob/main/IAGCN_Seemval14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("charitarth/semeval-2014-task-4-aspectbasedsentimentanalysis")

print("Path to dataset files:", path)

100%|██████████| 954k/954k [00:00<00:00, 27.5MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/charitarth/semeval-2014-task-4-aspectbasedsentimentanalysis/versions/4


In [2]:
import os

# Path to the dataset
dataset_path = '/root/.cache/kagglehub/datasets/charitarth/semeval-2014-task-4-aspectbasedsentimentanalysis/versions/4'

# List the files in the dataset directory
os.listdir(dataset_path)


['Restaurants_Train_v2.csv',
 'Laptop_Train_v2.csv',
 'Restaurants_Test_Data_PhaseB.csv',
 'Restaurants_Train_v2.xml',
 'laptops-trial.csv',
 'laptops-trial.xml',
 'SemEvalSchema.xsd',
 'Restaurants_Test_Data_PhaseA.xml',
 'Laptops_Test_Data_PhaseA.csv',
 'restaurants-trial.xml',
 'restaurants-trial.csv',
 'Restaurants_Train.xml',
 'Laptops_Test_Data_PhaseB.csv',
 'Laptop_Train_v2.xml',
 'Laptops_Test_Data_PhaseA.xml',
 'Restaurants_Test_Data_PhaseA.csv',
 'Restaurants_Test_Data_phaseB.xml',
 'Laptops_Test_Data_phaseB.xml']

In [ ]:
import os

# List the files in the downloaded dataset directory
dataset_files = os.listdir(path)
print("Files in the dataset directory:")
for file in dataset_files:
    print(file)


Files in the dataset directory:
Restaurants_Train.xml
restaurants-trial.xml
Laptops_Test_Data_PhaseB.csv
Restaurants_Test_Data_phaseB.xml
Restaurants_Train_v2.csv
Laptops_Test_Data_PhaseA.csv
Restaurants_Test_Data_PhaseA.xml
laptops-trial.xml
SemEvalSchema.xsd
Laptop_Train_v2.csv
Laptop_Train_v2.xml
Restaurants_Train_v2.xml
laptops-trial.csv
restaurants-trial.csv
Laptops_Test_Data_phaseB.xml
Restaurants_Test_Data_PhaseB.csv
Laptops_Test_Data_PhaseA.xml
Restaurants_Test_Data_PhaseA.csv


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
train_file = os.path.join(path, 'Laptop_Train_v2.csv')  # Adjust the filename if needed
test_file = os.path.join(path, 'Laptops_Test_Data_PhaseA.csv')    # Adjust the filename if needed

train_data = pd.read_csv(train_file)
test_data = pd.read_csv(test_file)

# Check the first few rows of the data
print(train_data.head())


     id                                           Sentence     Aspect Term  \
0  2339  I charge it at night and skip taking the cord ...            cord   
1  2339  I charge it at night and skip taking the cord ...    battery life   
2  1316  The tech guy then said the service center does...  service center   
3  1316  The tech guy then said the service center does...    "sales" team   
4  1316  The tech guy then said the service center does...        tech guy   

   polarity  from   to  
0   neutral    41   45  
1  positive    74   86  
2  negative    27   41  
3  negative   109  121  
4   neutral     4   12  


In [4]:
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

# Download NLTK stopwords if you haven't already
nltk.download('stopwords')
from nltk.corpus import stopwords

# Load the dataset
train_data = pd.read_csv('/root/.cache/kagglehub/datasets/charitarth/semeval-2014-task-4-aspectbasedsentimentanalysis/versions/4/Laptop_Train_v2.csv')

# Function for cleaning the text
def clean_text(text):
    # Remove special characters, digits, and extra spaces
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Clean the 'Sentence' column
train_data['cleaned_sentence'] = train_data['Sentence'].apply(clean_text)

# Tokenize the text using BERT tokenizer (for later usage with BERT-based models)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenizing the cleaned sentences
train_data['tokenized_sentence'] = train_data['cleaned_sentence'].apply(lambda x: tokenizer.tokenize(x))

# Sentiment label encoding: positive=2, neutral=1, negative=0
sentiment_mapping = {'positive': 2, 'neutral': 1, 'negative': 0}
train_data['sentiment_label'] = train_data['polarity'].map(sentiment_mapping)

# Inspecting the preprocessed data
print(train_data[['Sentence', 'cleaned_sentence', 'tokenized_sentence', 'sentiment_label']].head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

                                            Sentence  \
0  I charge it at night and skip taking the cord ...   
1  I charge it at night and skip taking the cord ...   
2  The tech guy then said the service center does...   
3  The tech guy then said the service center does...   
4  The tech guy then said the service center does...   

                                    cleaned_sentence  \
0  I charge it at night and skip taking the cord ...   
1  I charge it at night and skip taking the cord ...   
2  The tech guy then said the service center does...   
3  The tech guy then said the service center does...   
4  The tech guy then said the service center does...   

                                  tokenized_sentence  sentiment_label  
0  [i, charge, it, at, night, and, skip, taking, ...              1.0  
1  [i, charge, it, at, night, and, skip, taking, ...              2.0  
2  [the, tech, guy, then, said, the, service, cen...              0.0  
3  [the, tech, guy, then, said, the, s

In [6]:
from transformers import BertTokenizer
import pandas as pd
from tqdm import tqdm

# Load tokenizer (you can use 'roberta-base' or any other if you prefer)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


# For storing processed results
input_ids = []
attention_masks = []
token_type_ids = []

# Loop through each row
for _, row in tqdm(train_data.iterrows(), total=len(train_data)):
    sentence = row['Sentence']
    aspect = row['Aspect Term']

    # Format input as: [CLS] sentence [SEP] aspect [SEP]
    encoded = tokenizer.encode_plus(
        text=sentence,
        text_pair=aspect,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_token_type_ids=True,
        return_attention_mask=True,
        return_tensors='pt'  # PyTorch tensors
    )

    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])
    token_type_ids.append(encoded['token_type_ids'])

print("Tokenization complete.")


100%|██████████| 2358/2358 [00:02<00:00, 877.01it/s]

Tokenization complete.


In [9]:
# View tokenization of first 5 examples
for i in range(20):
    print(f"\nExample {i+1}")
    print("Sentence:", train_data.iloc[i]['Sentence'])
    print("Aspect:", train_data.iloc[i]['Aspect Term'])

    # Decode tokens for readability
    tokens = tokenizer.convert_ids_to_tokens(input_ids[i][0])
    print("Tokens:", tokens)

    # Optional: view token type ids
    print("Token Type IDs:", token_type_ids[i][0].tolist())

    # Optional: attention mask
    print("Attention Mask:", attention_masks[i][0].tolist())



Example 1
Sentence: I charge it at night and skip taking the cord with me because of the good battery life.
Aspect: cord
Tokens: ['[CLS]', 'i', 'charge', 'it', 'at', 'night', 'and', 'skip', 'taking', 'the', 'cord', 'with', 'me', 'because', 'of', 'the', 'good', 'battery', 'life', '.', '[SEP]', 'cord', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PA

In [13]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import torch

# Assuming you have the following lists: input_ids, attention_masks, token_type_ids, labels
train_inputs, val_inputs, train_masks, val_masks, train_types, val_types, train_labels, val_labels = train_test_split(
    input_ids, attention_masks, token_type_ids, train_data['sentiment_label'].values, test_size=0.1, random_state=42
)

# Convert lists of tensors to single tensors
train_inputs = torch.cat(train_inputs, dim=0)
val_inputs = torch.cat(val_inputs, dim=0)
train_masks = torch.cat(train_masks, dim=0)
val_masks = torch.cat(val_masks, dim=0)
train_types = torch.cat(train_types, dim=0)
val_types = torch.cat(val_types, dim=0)

# Convert labels to tensors
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

# Create TensorDatasets
train_data = TensorDataset(train_inputs, train_masks, train_types, train_labels)
val_data = TensorDataset(val_inputs, val_masks, val_types, val_labels)

# Create DataLoaders
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=32)